# Graph Library

**THIS IS ONLY FOR 15-688 STUDENTS**

In this problem, you're going to write a (very minimal) graph library, which uses both the adjacency dictionary and the sparse adjacency matrix representation of a graph.  Using these two representations, you'll implement two of the more well-known large-scale graph processing algorithms: Djisktra's algorithm for finding single-source shortest paths in the graph, and the PageRank algorithm for determining the importance of nodes in the network.

### The `wikipedia_small` graph

The graph we'll be focusing on for most of this assignment is a directed graph that represents the page links in the English language Wikipedia.  Specifically, we took the (pre-processed) Wikipedia dump from here: http://haselgrove.id.au/wikipedia.htm , which were taken from a 2008 version of Wikipedia, and we then selected only subselected only those nodes that had at least _500 incoming links_.  This resulted in a graph with about 24,000 nodes and 6,000,000 edges.

There are two files included with this notebook that are relevant here:

- `wikipedia_small.graph.gz`
- `wikipedia_small.nodes.gz`
    
The `.graph.gz` file contains a (gzipped) list of integers, two per each line.  If the line "`i j`" appears in the file, this indicates a directed edge from node `i` to node `j`.  The `.nodes.gz` file then contains a (gzipped) list of each node the the graph, where the link number indicates the node index.  This is how we can relate the node numbers in the `.graph.gz` file to actual pages on Wikipedia.

Don't decompress the `.gz` files - we do that while reading them. This is common practice when dealing with large amounts of text data.

### Warmup using `networkx`

Bundled with this handout is `hw2_graph_library_warmup.ipynb`, which introduces you to the `networkx` library. We recommend (but not require) that you do that first. You are not allowed to use the `networkx` library in this assignment.

## Your own Graph class

In the main portion of this assignment, you'll create your own Graph class that mimics some of the functionality of networkx (and which will indeed be much faster than networkx when it comes to algorithms like PageRank). We'll provide you with some scaffolding and support code.

In [1]:
import numpy as np
import scipy.sparse as sp
import heapdict
import gzip
from collections import defaultdict
from testing.testing import test

SIMPLE_TEST_GRAPH = [("A","B"), ("B","C"), ("A","D"), ("D", "E"), ("E", "B")]

# Utility function to keep a copy of the list in memory:
global _files_cache
_files_cache = None
def test_graph():
    global _files_cache
    if _files_cache is None:
        _files_cache = read_files()
    # Return a generator so users can't mess with the cached file.
    # Avoids the overhead of copying the list:
    return (n for n in _files_cache)

# Utility function to read the edges:
def read_files(basename="wikipedia_small"):
    with gzip.open(f"{basename}.nodes.gz", 'rt',encoding='UTF-8') as f:
        nodes = [a.strip() for a in f]
    with gzip.open(f"{basename}.graph.gz", 'rt') as f:
        links = []
        for row in f:
            i, j = tuple(row.strip().split())
            links.append((nodes[int(i)], nodes[int(j)]))
    return links

### Template

Here is the template for your Graph class.

Note that `self.edges` should be represented as an "adjacency dictionary", so that for every node `i` in the graph `self.edges[i]` is a dictionary of nodes that `i` is directly connected to. The value of the inner dictionary should be `1` for every edge that exists and `0` otherwise. (The value of this entry doesn't matter, so we could technically use a dictionary of sets, but we use a dictionary of dictionaries to keep things a little bit more uniform and to allow for potential extensions e.g. to weighted graphs.)

Note that all vertices must exist in the dictionary. If a vertex has no outgoing edges, then it should still have an entry pointing to an empty dictionary. 

In [2]:
class Graph:
    def __init__(self, initial_edges=None):
        """ Initialize with an empty dictionary-of-sets. """
        self.edges = defaultdict(lambda: defaultdict(int))
        if initial_edges is not None:
            self.add_edges(initial_edges)

    def __len__(self):
        """ Number of edges in the graph """
        return sum(len(v) for v in self.edges.values())

    def add_edges(self, edges_list):
        """ Add a list of edges to the network. Use 1 to indiciate the presence of an edge. 

        Args:
            edges_list: list of (a, b) tuples, where a -> b is an edge to add
        """
#         print(edges_list)
        for source, dest in edges_list:
            self.edges[source][dest] = 1
            if (dest not in self.edges.keys()):
                self.edges[dest] = defaultdict(int)
#         print(self.edges)

    # We will implement these functions later:
    def shortest_path(self, source):
        return shortest_path_impl(self.edges, source)
    
    def adjacency_matrix(self):
        return adjacency_matrix_impl(self.edges)
    
    def pagerank(self, d=0.85, iters=100):
        return pagerank(self.adjacency_matrix(), d, iters)

## Q1: Implement `add_edges()` call

To begin, implement the function `add_edges()` in the code above. It must modify the `self.edges` variable to add all edges passed as tuples in `edges_list`.

We will test your `add_edges()` implementation on this graph:

    A -> B -> C
    |    ^
    v    |
    D -> E
    

In [4]:
def add_edges_impl_test(add_edges_impl):
    G = add_edges_impl(SIMPLE_TEST_GRAPH)
    
    # If this test fails, you need to make sure that nodes with no outgoing edges are present in G.edges:
    test.true("C" in G.edges)
    # If this test fails, you need to make sure that al values are `int(1)`.
    test.true(all(all(vv == 1 for vv in v.values()) for v in G.edges.values()))

    test.equal(set(G.edges.keys()), set("ABCDE"))
    test.equal(set(G.edges["A"].keys()), set("BD"))
    
    # Load the test graph. This may take a few seconds:
    G = add_edges_impl(test_graph())
    test.equal(len(G.edges), 24166)
    test.equal(len(G.edges["Carnegie_Mellon_University"]), 162)

@test
def add_edges_impl(edges):
    return Graph(edges)

### TESTING add_edges_impl: PASSED 6/6
###



## Q2: Djisktra's algorithm

Next, implement Djisktra's single-source shortest path algorithm (with the simple case where the distance along any edge is assumed to be one).  You can refer to the [Wikipedia page on Djikstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) for reference. The basic idea of the algorithm is to keep a priority queue of nodes ordered by distance from a source node.  At each step, we continually pop off the smallest element `i` in the queue, and update the distance of all successor nodes to have a distance of `1 + distance[i]`.

For the priority queue, you should use a [`heapdict`](https://github.com/DanielStutzbach/heapdict), which is a form of priority queue that allows you to change the priority of an element.

When called with source node `A`, the function should return a dictionary where the keys are all the nodes in the graph. For each key `B`:

- if `B` is reachable from `A` then the value must be the tuple `(distance, prev_node)`, where:
  - `distance` is the minimum number of hops from `A` to `B`, and
  - `prev_node` is the node immediately before `B` along one such shortest path
- if `B` is not reachable from `A`, then the value must be `None`
- if `B == A`, then the value should be `(0, None)`

In [5]:
import sys
from heapdict import heapdict

def shortest_path_impl_test(shortest_path_impl):
    G = Graph(SIMPLE_TEST_GRAPH)
    sp = shortest_path_impl(G.edges, "A")
    # Check that you correctly handle `B==A`:
    test.equal(sp['A'], (0, None))
#     Simple test:
    test.equal(sp, {'A': (0, None), 'B': (1, 'A'), 'C': (2, 'B'), 'D': (1, 'A'), 'E': (2, 'D')})
    
    # Check that you correctly handle unreachable nodes:
    test.equal(shortest_path_impl(G.edges, "B"), {'A': None, 'B': (0, None), 'C': (1, 'B'), 'D': None, 'E': None})

    # Load the test graph. This should take <15s:
    G = add_edges_impl(test_graph())
    sp = shortest_path_impl(G.edges, "Carnegie_Mellon_University")
    test.equal(sp["Singapore"], (1, "Carnegie_Mellon_University"))
    test.equal(sp["Data"][0], 2)
    test.equal(sp["List_of_Salticidae_species"][0], 5)
    # Unreachable nodes
    test.equal(sp['Village_Development_Committee_(Nepal)'], None)
    test.equal(sp['Voice_acting_in_Japan'], None)
    
    # The maximum distance is 5. We determined this in the warmup:
    test.equal(max(*(d for d,_ in filter(lambda x: x, sp.values()))), 5)


@test
def shortest_path_impl(edges, source):
    """ Compute the single-source shorting path.

    This function uses Djikstra's algorithm to compute the distance from 
    source to all other nodes in the network.

    Args:
        source: node index for the source

    Returns: Dict[str, Optional[Tuple[int, str]]]
        dictionary of node: (distance, prev_node) values for each reachable node in the graph, where 
              distance denotes the shortest distance from of node from source, and
              prev_node is the previous node on the shortest path from source to node.
            if node is unreachable, the value should be None
    """
    
    dist = heapdict()
    prev = dict()
    result = dict()
    for n in edges.keys():
#         print(n)
        dist[n] = sys.maxsize
        prev[n] = None
    
    dist[source] = 0
    
    while(len(dist) != 0):
        node, distance = dist.popitem()
        for neighbor in edges[node]:
            if neighbor in dist.keys():
                alt = distance + 1
                if (alt < dist[neighbor]):
                    dist[neighbor] = alt
                    prev[neighbor] = node
        if (prev[node] == None and distance != 0):
            result[node] = None
        else:
            result[node] = (distance, prev[node])
    return result

### TESTING shortest_path_impl: PASSED 9/9
###



## Q3: Adjacency matrix representation

Implement the adjacency matrix method of the Graph class.  This returns a matrix representing the adjacency of the graph (in scipy COO sparse format), as well as a list of nodes that indicate how the indices in this graph relate to the nodes in the network.

In order to complete this question in a manner that works on the Wikipedia, you must implement this function natively as a sparse matrix (i.e., you cannot construct a dense matrix and then convert that to a sparse matrix, but need to directly use the `sp.coo_matrix()` constructor).  The Wikipedia graph is is 24K x 24K nodes, which (assuming 8 bytes per entry, would take up 4GB of memory.  While it's not impossible to do things this way at this scale (it quickly becomes infeasible for graphs that are even slightly larger), it's a very bad idea, and just allocating this much memory will take too long.

**Note the order of the axes of the output matrix.** This is important for calculating the PageRank.

In [6]:
from scipy.sparse import coo_matrix

def adjacency_matrix_impl_test(adjacency_matrix_impl):
    def check(graph):
        G = Graph(graph)
        adj, nodes = adjacency_matrix_impl(G.edges)
        # Check output type:
        test.true(isinstance(adj, sp.coo_matrix))
        test.equal(str(adj.dtype), "uint8")
        # Make sure there are the correct number of entries:
        test.equal(adj.sum(), len(G))
        # Make sure that every entry is represented:
        lookup = { node : i for i, node in enumerate(nodes) }
        adj_lil = adj.tolil() # Convert to linked-list sparse matrix for lookups:
        test.true(all(adj_lil[lookup[j], lookup[i]]==1 for i, j in graph))

    check(SIMPLE_TEST_GRAPH)
    check(test_graph())

@test
def adjacency_matrix_impl(edges):
    """ Compute an adjacency matrix form of the graph.

    Returns: tuple(A, nodes)
        A : sp.coo_matrix[np.uint8] -- a sparse adjacency matrix with underlying `dtype=np.uint8` that represents the graph (i.e., `A[j,i] == 1` iff there is an edge i->j)
           NOTE: be sure you have this ordering correct!
        nodes : List[str] -- a list of nodes indicating the key corresponding to each index of the `A` matrix
    """
    List = [k for k in edges.keys()]
    Dict = {List[index]:index for index in range(0,len(List))}
    row = []
    col = []

    for source in edges.keys(): 
        for dest in edges[source].keys():
            row.append(Dict[dest])
            col.append(Dict[source])

    data = [1 for _ in range(len(row))]
    A = coo_matrix((data,(row,col)), dtype=np.uint8, shape = (len(List),len(List)))
    
    return (A, List)

### TESTING adjacency_matrix_impl: PASSED 8/8
###



Make sure your code works on the Wikipedia graph.  In our implementation, it takes 7 seconds to run all tests.

## Q4: PageRank algorithm

Finally, implement the PageRank algorithm using the adjacency matrix representation. You should use the approach described in the ["Power Method" section on the Wikipedia entry](https://en.wikipedia.org/wiki/PageRank#Power_Method), which we also discussed in class.

This involves forming some initial uniform probability vector $x$, and repeatly multiplying it by the matrices:
\begin{equation}
x \gets \left(d P + \left(\frac{1-d}{n}\right) E \right)x
\end{equation}
where $P$ is a transition matrix, $E$ is the matrix of all ones, and $d$ is the damping factor. You get $P$ by normalizing $A$ so that all columns have sum 1.

Recall that from the definition of PageRank, when we reach a "sink" node (a node with no outgoing edges), we randomly hop to any other node in the network, so that columns of $P$ that have no outgoing edges are set to the uniform distribution.  To be efficient, you'll also want to avoid explicitly forming the $E$ matrix, and should instead use the fact that $E = 1*1^T$ where $1$ denotes a vector of all ones.  Use the fact that we can reorder matrix multiplication if associative (i.e., the fact the $A(BC)$ = $(AB)C$) to make this operation as fast as possible.

Your function should return a dictionary of nodes and their corresponding page rank.  For example, in our graph above, we have the following results:
```
C: 0.324
B: 0.281
E: 0.188
D: 0.121
A: 0.085
```
As is intuitive, nodes B and C have higher page rank, as they are pointed to by more of the other nodes.

In [209]:
def pagerank_impl_test(pagerank_impl):
    G = Graph(SIMPLE_TEST_GRAPH)
    pagerank = pagerank_impl(G.adjacency_matrix())    
    test.true(abs(pagerank['A']-0.08510862387068166) < 1e-7)
    test.true(abs(pagerank['B']-0.28124676686965944) < 1e-7)
    test.true(abs(pagerank['C']-0.3241683757098922) < 1e-7)
    test.true(abs(pagerank['D']-0.12127978901572137) < 1e-7)
    test.true(abs(pagerank['E']-0.18819644453404483) < 1e-7)

    G = Graph(test_graph())
    pagerank = pagerank_impl(G.adjacency_matrix())

    test.equal(len(pagerank), 24166)

    # Numerical check
    test.true(abs(pagerank['United_States'] - 0.00275188705264) < 1e-5)
    test.true(abs(pagerank['2008'] - 0.00217342514773) < 1e-5)
    test.true(abs(pagerank['Canada'] - 0.00109896195215) < 1e-5)
    test.true(abs(pagerank['World_War_II'] - 0.00104913079624) < 1e-5)
    test.true(abs(pagerank['List_of_African_films'] - 0.00100713870383) < 1e-5)
    test.true(abs(pagerank['Europe'] - 0.000937690025073) < 1e-5)
    test.true(abs(pagerank['English_language'] - 0.000908144359626) < 1e-5)
    test.true(abs(pagerank['Geographic_coordinate_system'] - 0.000891711151403) < 1e-5)
    test.true(abs(pagerank['Latin'] - 0.000888662228804) < 1e-5)

@test
def pagerank_impl(adjacency_matrix, d=0.85, iters=100):
    """ Compute the PageRank score for each node in the network.

    Compute PageRank scores using the power method.

    Args:
        adjacency_matrix: Tuple[A, nodes] -- output from Graph.adjacency_matrix()
    
    Keyword Args:
        d: 1 - random restart factor
        iters: maximum number of iterations of power method

    Returns: dict ranks
        ranks: a dictionary of node:importance score, for each node in the
               network (larger score means higher rank)

    """
    A, nodes = adjacency_matrix
    n,_ = A.shape
    sums = A.sum(0).T          
    B = A.tocsr()

    r = np.ones((n,1))/n
    for _ in range(0,iters):
        with np.errstate(divide='ignore'):
            r1 = B.dot(r*d/sums)
        r1 = r1 + (1 - r1.sum())/n
        r = r1
#     print(r)
    Dict = {}
    for i in range(0, len(r)):
        Dict[nodes[i]] = float(r[i])
#     print(Dict)
    return Dict
    
# Xindi Lan explained to me the basic idea of solving Q4: PageRank algorithm

### TESTING pagerank_impl: PASSED 15/15
###



Make sure your implementation works on the full Wikipedia graph (in our implementation, it takes 11 seconds to run, most of which is taken up by generating the adjacency matrix).  The top PageRank entires we get from our implementation are:

```
United_States     2.75e-3
2007              2.44e-3
2008              2.17e-3
Wikimedia_Commons 1.72e-3
United_Kingdom    1.59e-3
2006              1.54e-3
France            1.44e-3
Wiktionary        1.26e-3
Canada            1.09e-3
World_War_II      1.04e-3
2005              1.04e-3
List_of_Africa... 1.00e-3
Germany           0.95e-3
Europe            0.93e-3
English_language  0.90e-3
Geographic_coo... 0.89e-3
Latin             0.88e-3
Australia         0.87e-3
India             0.78e-3
Japan             0.78e-3
```

countries and years! A seemingly reasonable list of pages we may expect to be important.